# Setup

In [35]:
%%capture
!pip install transformers
!pip install datasets

In [36]:
import numpy as np
import sklearn
import pandas as pd
import tensorflow as tf

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# constants and global variables
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CIL/Dataset/{}'
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/CIL/Models/{}'
PROBABILITIES = '/content/drive/MyDrive/Colab Notebooks/CIL/Probabilities/{}'
PREDICTIONS = '/content/drive/MyDrive/Colab Notebooks/CIL/Predictions/{}'

# MODEL_NAME = "bert-base-cased"
# MODEL_NAME =  "roberta-base"
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"

# if set to true the trainig of the classifier models will be performed, otherwise the 
# models will be loaded from a file (if present)
is_train_enabled = True

# Read Preprocessed Data

In [39]:
def load_X_data(file_name):
  tweets = []
  with open(DATA_PATH.format(file_name), 'r', encoding='utf-8') as f:
      for line in f:
        tweets.append(line.rstrip())
  
  return np.array(tweets)

X_train = load_X_data("X_train_processed_bert_full.txt")
X_test = load_X_data("X_test_processed_bert_full.txt")

# swap url and user with @user and http
if MODEL_NAME == "cardiffnlp/twitter-roberta-base-sentiment-latest":
  X_train = list(map(lambda tweet : tweet.replace("<user>", "@user").replace("<url>", "http"), X_train))
  X_test = list(map(lambda tweet : tweet.replace("<user>", "@user").replace("<url>", "http"), X_test))

In [40]:
def load_y_data(file_name):
  labels = []
  with open(DATA_PATH.format(file_name), 'r', encoding='utf-8') as f:
      for line in f:
        labels.append(int(line.rstrip()))
  
  return np.array(labels)

y_train = load_y_data("y_train_full.txt")

In [41]:
# X_train = X_train[0:500000]
# y_train = y_train[0:500000]
for i in range(10):
  print(X_train[i])
  print(X_test[i])
  print(y_train[i])

@user yes i didn't even realize he did ! i really wanted him to respond because i want to buy him candy or something
sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air , stay longer in the water and ... http
1
bradly james lowrey is my bestfriend & & & he mean alot to me #justthoughtidleteveryoneknow
@user shucks well i work all week so now i can't come cheer you on ! oh and put those batteries in your calculator ! ! !
1
mckleinusa ashburn 15144 s series leather laptop case ( brown clean , front flap-over design with a secure key l ... http
i cant stay away from bug thats my baby
0
@user next time ima come in yo class nd wake you up myself ! i wanted you to come out , so i can get my hug but you was sleep
@user no ma'am ! ! ! lol im perfectly fine and not contagious anymore lmao
0
trivial pursuit for juniors ( second edition the second edition for juniors of the legendary trivial pursuit games http
whenever i fall asleep watching the tv , i always wake up wit

In [42]:
from sklearn.model_selection import train_test_split

# split the model into training test and validation
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=55)

# BERT HUGGINFACE 

In [43]:
from datasets import Dataset

# go from list type to 'Dataset' type. This object is requiered to train the model
df = pd.DataFrame(X_train, columns =['Phrase'])
train = Dataset.from_pandas(df).add_column(name="Label", column=y_train)

# df = pd.DataFrame(X_val, columns =['Phrase'])
# val = Dataset.from_pandas(df).add_column(name="Label", column=y_val)

df = pd.DataFrame(X_test, columns =['Phrase'])
test = Dataset.from_pandas(df)

In [44]:
# perform tokenization of the dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(sentence):
    return tokenizer(sentence['Phrase'], padding=True, truncation=True, max_length=40)

train = train.map(tokenize_function, batched=True)
test = test.map(tokenize_function, batched=True)
# val = val.map(tokenize_function, batched=True)

for i in range(5):
  print(train[i])
  print(test[i])
  # print(val[i])

  0%|          | 0/2375 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/125 [00:00<?, ?ba/s]

{'Phrase': '@user damn well a pic like that proves why your mother should have swallowed you instead', 'Label': 1, 'input_ids': [0, 1039, 12105, 16490, 157, 10, 1893, 101, 14, 14476, 596, 110, 985, 197, 33, 32305, 47, 1386, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'Phrase': 'sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air , stay longer in the water and ... http', 'input_ids': [0, 16466, 109, 139, 1759, 3342, 2850, 15917, 36, 1612, 19, 5, 15295, 3342, 12, 417, 3036, 842, 8631, 22923, 4097, 935, 2156, 1095, 1181, 11, 5, 514, 8, 1666, 2054, 2, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]}
{'Phrase': "@user believe you have some homework reading your going to teach me mr . smarty

In [45]:
# go from 'Dataset' type to tensorflow so that our dataset can be used for training in keras
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

tf_train_dataset = train.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["Label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=256,
)

# tf_val_dataset = val.to_tf_dataset(
#     columns=["attention_mask", "input_ids", "token_type_ids"],
#     label_cols=["Label"],
#     shuffle=False,
#     collate_fn=data_collator,
#     batch_size=256,
# )

tf_test_dataset = test.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=256,
)

In [46]:
from transformers import TFAutoModelForSequenceClassification

# Compute some variables needed to speed up training
batch_size = 64
train_steps_per_epoch = int(len(tf_train_dataset) // batch_size)
# dev_steps_per_epoch = int(len(tf_val_dataset) // batch_size)


# download pre-trained model
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
if is_train_enabled:
  # train model
  model.fit(tf_train_dataset,
            # validation_data=tf_val_dataset,
            epochs=1,
            verbose=2,
            # steps_per_epoch=train_steps_per_epoch,
            # validation_steps=dev_steps_per_epoch,
            )
  
  # save model to file
  model.save_pretrained(MODEL_PATH.format(MODEL_NAME.replace("/", "_")))

9278/9278 - 17435s - loss: 0.2442 - sparse_categorical_accuracy: 0.8968 - val_loss: 0.2274 - val_sparse_categorical_accuracy: 0.9062 - 17435s/epoch - 2s/step


In [48]:
# load trained model from file
model.load_weights(MODEL_PATH.format(MODEL_NAME.replace("/", "_") + "/tf_model.h5"))

In [49]:
from scipy.special import softmax

y_pred = model.predict(tf_test_dataset)[0]
y_pred = list(map(lambda x : softmax(x), y_pred))

# write prediction probabilites to a file. These probabilites will be used to create
# an ensemble of models
def write_output_probas(file_name, Y):
  f  = open(file_name, "w")
  f.write("Id,Prediction\n")
  id = 1
  for y in Y:
    f.write(str(id) + "," + str(list(y)) + "\n")
    id = id + 1
  f.close()

write_output_probas(PROBABILITIES.format(MODEL_NAME.replace("/", "_") + "_predictions_probas.csv"), y_pred)


# write predictions to file
y_pred = np.argmax(np.array(y_pred), axis=-1)
def write_output(file_name, Y):
  f  = open(file_name, "w")
  f.write("Id,Prediction\n")
  id = 1
  for y in Y:
    if y == 0:
      y = -1
    f.write(str(id) + "," + str(y) + "\n")
    id = id + 1
  f.close()

write_output(PREDICTIONS.format(MODEL_NAME.replace("/", "_") + "_predictions.csv"), y_pred)